In [2]:
from google.colab import drive
drive.mount("/content/drive")
!ls

Mounted at /content/drive
drive  sample_data


In [ ]:
import os 
import numpy as  np
import pandas as pd
path="drive/MyDrive/Dong-Data"

In [ ]:
data=pd.read_csv(path+"/FinalExtract"+str(0)+".csv",  encoding = 'unicode_escape')
print(data.columns)

In [5]:
data.columns

Index(['Compustat GVKEY', 'Company Name_x', 'Director Network Size',
       'Time in Role', 'TICKER', 'InterlockingTies', 'Time on Board',
       'Total Number of Other Boards to Date', 'Age (Yrs)',
       'Number of Qualifications', 'Total Directors on the Board',
       'SIC Codes (Primary Code Only)', 'Total Assets',
       'Long-Term Debt [CY 1990] ($USDmm, Historical rate)',
       'Total Common Equity', 'Market Capitalization',
       'R&D Exp. [CY 1990] ($USDmm, Historical rate)', 'YEAR', 'size',
       'Growth', 'Sector', 'FIRM ID', 'GVKEY', 'GVKEY_YEAR', 'TICKER_x',
       'CUSIP', 'SIC', 'sumnew', 'new', 'year', 'permno', 'patents', 'citsum',
       'PERMNO', 'TICKER_y', 'leverage', 'R&D', 'logR&D', 'logSumnew',
       'logPatents', 'logCitsum'],
      dtype='object')

In [6]:
data.describe()

,Compustat GVKEY,Director Network Size,Time in Role,InterlockingTies,Time on Board,Total Number of Other Boards to Date,Age (Yrs),Number of Qualifications,Total Directors on the Board,SIC Codes (Primary Code Only),Total Assets,"Long-Term Debt [CY 1990] ($USDmm, Historical rate)",Total Common Equity,Market Capitalization,"R&D Exp. [CY 1990] ($USDmm, Historical rate)",YEAR,size,Growth,FIRM ID,GVKEY,GVKEY_YEAR,SIC,sumnew,new,year,permno,patents,citsum,PERMNO,leverage,R&D,logR&D,logSumnew,logPatents,logCitsum
count,42600.000000,42600.000000,42467.000000,7420.000000,42467.000000,7420.000000,42331.000000,42467.000000,42600.000000,42600.000000,4.260000e+04,42344.000000,42600.000000,42600.000000,42425.000000,42600.000000,42600.000000,42600.000000,42600.000000,42600.000000,4.260000e+04,42600.000000,42600.000000,42600.000000,42600.000000,42600.000000,42600.000000,42600.000000,42600.000000,42344.000000,42425.000000,42425.000000,42600.000000,42600.000000,42600.000000
mean,69298.445023,2280.998873,5.571872,0.145057,7.272039,1.354313,60.575630,2.186639,10.712582,4888.940869,4.788616e+04,8884.095232,10059.634440,27745.384854,184.827544,2013.372770,9.664037,2.554861,546.996197,23880.225704,2.388043e+08,3894.291948,0.962723,0.797441,2013.372770,47892.253920,167.568685,526.379272,47892.253920,0.249865,0.014486,0.013685,0.346406,3.085373,3.260524
std,84672.372403,2636.223460,5.077684,0.077943,6.378206,0.630525,8.566046,1.091779,3.151521,1646.904301,1.082406e+05,19307.349437,15596.694529,44620.018753,775.059472,3.250385,1.490323,12.373380,401.818063,38300.279011,3.830028e+08,1625.495105,2.901664,2.841697,3.250385,28138.107962,613.503852,2428.549046,28138.107962,0.213907,0.040179,0.036310,0.643151,1.831176,2.463441
min,1045.000000,0.000000,0.000000,0.031250,0.000000,1.000000,31.000000,0.000000,3.000000,1000.000000,2.090000e+01,0.000000,-85560.000000,62.500000,0.000000,2008.000000,3.039749,0.817427,5.000000,1078.000000,1.078202e+07,1311.000000,0.000000,0.000000,2008.000000,10107.000000,1.000000,0.000000,10107.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.000000
25%,7970.000000,640.000000,1.900000,0.083333,2.700000,1.000000,54.000000,2.000000,9.000000,3721.000000,5.321000e+03,1133.100000,1676.000000,7573.100000,0.000000,2011.000000,8.579417,1.230726,172.000000,5046.000000,5.046201e+07,2851.000000,0.000000,0.000000,2011.000000,18411.000000,4.000000,2.000000,18411.000000,0.113565,0.000000,0.000000,0.000000,1.609438,1.098612
50%,17874.000000,1545.000000,4.300000,0.125000,6.000000,1.000000,61.000000,2.000000,11.000000,4911.000000,1.410470e+04,2875.000000,4648.000000,14269.200000,0.000000,2014.000000,9.554263,1.651136,428.000000,9465.000000,9.465201e+07,3621.000000,0.000000,0.000000,2014.000000,45356.000000,16.000000,22.000000,45356.000000,0.237606,0.000000,0.000000,0.000000,2.833213,3.135494
75%,147579.000000,3020.000000,7.600000,0.166667,9.600000,2.000000,67.000000,3.000000,12.000000,6200.000000,4.657700e+04,10496.000000,13575.000000,28340.924757,0.000000,2016.000000,10.748862,2.407039,1014.000000,24135.000000,2.413520e+08,3841.000000,1.000000,0.000000,2016.000000,77178.000000,66.000000,131.000000,77178.000000,0.352321,0.000000,0.000000,0.693147,4.204693,4.882802
max,260774.000000,24251.000000,50.900000,0.666667,62.800000,4.000000,92.000000,11.000000,33.000000,8731.000000,1.930115e+06,267055.000000,184089.000000,860882.500000,14731.000000,2018.000000,14.473090,1086.331209,1161.000000,180405.000000,1.804052e+09,9997.000000,41.000000,41.000000,2018.000000,91733.000000,7502.000000,31440.000000,91733.000000,3.834362,0.947776,0.666688,3.737670,8.923058,10.355868


In [7]:
colName=['Number of Qualifications','Time in Role','Director Network Size','InterlockingTies','size','leverage','Growth','Age (Yrs)','Time on Board','R&D']
data=data[colName]

In [8]:
data.columns

Index(['Number of Qualifications', 'Time in Role', 'Director Network Size',
       'InterlockingTies', 'size', 'leverage', 'Growth', 'Age (Yrs)',
       'Time on Board', 'R&D'],
      dtype='object')

In [9]:
def replaceNaNWithMean(data,col):
  #tinh mean cua rieng cot col
  mean=np.mean(data[col])
  #thay gia tri NaN cua cot col bang mean cua cot do
  data[col].fillna(mean, inplace=True)

In [10]:
for col in colName:
  replaceNaNWithMean(data,col)

In [11]:
split=int(0.8*len(data))

In [23]:
len(data)

42600

In [12]:
split

34080

In [13]:
#Create training features and labels for training dataset
train_X=data.iloc[:split,0:data.shape[1]-1].values
test_X=data.iloc[split:,0:data.shape[1]-1].values
#Create training features and labels for test dataset
train_y=data.iloc[:split,data.shape[1]-1].values
test_y=data.iloc[split:,data.shape[1]-1].values

In [14]:
print(train_X.shape)
print(test_X.shape)
print(train_y.shape)
print(test_y.shape)

(34080, 9)
(8520, 9)
(34080,)
(8520,)


In [15]:
print(pd.DataFrame(train_y).isnull().values.any())
print(pd.DataFrame(test_y).isnull().values.any())

False
False


In [16]:
print(pd.DataFrame(train_X).isnull().values.any())
print(pd.DataFrame(test_X).isnull().values.any())

False
False


In [17]:
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.svm import SVR
def svm(train_X, train_y,test_X,test_y):
  for kernel in ["rbf","linear","poly", "sigmoid"]:
    for gamma in [0.1, 0.01, 0.001]:
      for c in [1, 0.1, 0.01]:
        for error in ["mse", "mae"]:
          regressor = SVR(kernel = kernel, gamma=gamma, C=c)
          #training
          regressor.fit(train_X, train_y)

          #predict on unseen data
          y_pred = regressor.predict(test_X)
          
          if error=="mse":
            #calculate mean squared error
            er = mean_squared_error(test_y, y_pred)
          else:
            er= mean_absolute_error(test_y, y_pred)

          #display mse
          print("with kernel ", kernel, " gamma ", gamma, " c ", c, " error ", error, " = ", er)
          # acc=0
          # for i in range(len(y_pred)):
          #   # print("test_y[i], y_pred[i] ",test_y[i]," ",y_pred[i],"accuracy ",1-math.fabs(test_y[i]-y_pred[i])/test_y[i])
          #   acc=acc+1-math.fabs(test_y[i]-y_pred[i])/test_y[i]
          # print("average acc ",acc/len(y_pred))

In [ ]:
svm(train_X, train_y,test_X,test_y)

In [19]:
regressor = SVR(kernel = "rbf", gamma=0.1, C=1)
#training
regressor.fit(train_X, train_y)

#predict on unseen data
y_pred = regressor.predict(test_X)

In [ ]:
f=open(path+"/Prediction.csv","w")
f.write("Prediction,True value\n")
for i,j in zip(y_pred,test_y):
  print(i," ",j)
  f.write(str(i)+","+str(j)+"\n")
f.close()